In [239]:
import pandas as pd #dataframes
import matplotlib.pyplot as plt 
import numpy as np # n-dim object support
# do ploting inline instead of in a separate window
%matplotlib inline
import random

In [240]:
df_school = pd.read_csv("../Scraper/school_records.csv")
df_ps_game = pd.read_csv("../Scraper/post_season_game_records.csv")

In [241]:
df_school.shape

(3478, 21)

In [242]:
df_ps_game.shape

(504, 7)

In [243]:
df_school.head(5)

,year,team_name,fg_pg,ft_pg,three_pt_pg,orb_pg,drb_pg,ast_pg,stl_pg,blk_pg,...,pf_pg,pt_pg,opnt_pt_pg,fg_pct,three_p_pct,ft_pct,wl_pct,conf_wl_pct,srs,sos
0,2010,Air Force,20.387097,10.741935,5.677419,7.096774,27.222685,12.548387,5.000000,1.645161,...,17.645161,57.193548,63.129032,0.443,0.313,0.635,0.323,0.062500,-4.90,3.13
1,2010,Akron,25.057143,13.800000,6.714286,13.342857,35.875918,13.514286,6.085714,3.257143,...,19.485714,70.628571,65.514286,0.433,0.339,0.657,0.686,0.750000,2.82,-1.50
2,2010,Alabama A&M,22.185185,17.481481,4.000000,13.925926,36.669410,10.666667,9.222222,5.296296,...,20.370370,65.851852,69.666667,0.382,0.291,0.635,0.407,0.444444,-20.19,-13.71
3,2010,Alabama-Birmingham,22.441176,16.852941,5.205882,12.352941,36.342561,11.470588,6.558824,2.676471,...,17.970588,66.941176,60.382353,0.422,0.311,0.694,0.735,0.687500,9.46,2.90
4,2010,Alabama State,21.516129,15.290323,6.129032,12.903226,35.099896,12.903226,7.354839,4.161290,...,20.451613,64.451613,65.903226,0.404,0.324,0.641,0.516,0.666667,-14.41,-12.02


In [244]:
df_ps_game.head(5)

,year,team_1_name,team_1_score,team_1_seed,team_2_name,team_2_score,team_2_seed
0,2011,UTSA,46,16,Ohio State,75,1
1,2011,George Mason,61,8,Villanova,57,9
2,2011,Clemson,76,12,West Virginia,84,5
3,2011,Princeton,57,13,Kentucky,59,4
4,2011,Marquette,66,11,Xavier,55,6


In [245]:
df_school.isnull().values.any()

False

In [246]:
df_ps_game.isnull().values.any()

False

In [247]:
import shared
ps_team_stats_df = shared.get_team_stats_df(df_school, df_ps_game, True)

2018
UNC Greensboro
2018
Penn
2018
NC State
2018
TCU
2018
UMBC
2018
UMBC
498


In [248]:
ps_game_w_team_stats = pd.concat([df_ps_game, ps_team_stats_df], axis=1, join='inner')

In [249]:
ps_game_w_team_stats.head(3)

,year,team_1_name,team_1_score,team_1_seed,team_2_name,team_2_score,team_2_seed,team_name_1,fg_pg_1,ft_pg_1,...,pt_pg_2,opnt_pt_pg_2,fg_pct_2,three_p_pct_2,ft_pct_2,wl_pct_2,conf_wl_pct_2,srs_2,sos_2,t1_win
0,2011,UTSA,46,16,Ohio State,75,1,Texas-San Antonio,23.588235,16.058824,...,77.135135,59.675676,0.494,0.423,0.701,0.919,0.888889,25.84,8.38,False
1,2011,George Mason,61,8,Villanova,57,9,George Mason,25.764706,14.558824,...,72.242424,65.424242,0.438,0.348,0.757,0.636,0.500000,15.05,8.23,True
2,2011,Clemson,76,12,West Virginia,84,5,Clemson,23.823529,14.500000,...,69.787879,64.666667,0.429,0.337,0.711,0.636,0.611111,16.15,11.03,False


In [250]:
ps_game_w_team_stats.shape

(498, 48)

## Check team 1 winning true/false ratio

In [251]:
t1_win_map = {True:1, False:0}
ps_game_w_team_stats['t1_win'] = ps_game_w_team_stats['t1_win'].map(t1_win_map)
num_true = len(ps_game_w_team_stats.loc[ps_game_w_team_stats['t1_win'] == True])
num_false = len(ps_game_w_team_stats.loc[ps_game_w_team_stats['t1_win'] == False])
print("Number of True cases: {0} ({1:2.2f}%)".format(num_true, (num_true/(num_true+num_false))*100))
print("Number of False cases: {0} ({1:2.2f}%)".format(num_false, (num_false/(num_true+num_false))*100))

Number of True cases: 259 (52.01%)
Number of False cases: 239 (47.99%)


In [252]:
def scale_features(data, col_names):
    scaled_features = {}
    for col_name in col_names:
        mean, std = data[col_name].values.mean(), data[col_name].values.std()
        scaled_features[col_name] = [mean, std]
        data.loc[:, col_name] = (data[col_name].values - mean)/std
    return scaled_features


In [253]:
import sklearn.model_selection
scale_features(ps_game_w_team_stats, shared.ps_feature_col_names)
ps_x = ps_game_w_team_stats[shared.ps_feature_col_names].values
ps_y = ps_game_w_team_stats[shared.ps_predict_class_names].values
print(type(ps_x))
split_test_size = 0.2
ps_x_train, ps_x_test, ps_y_train, ps_y_test = sklearn.model_selection.train_test_split(ps_x, ps_y, test_size=split_test_size, random_state=42)
split_valid_size = 0.25
ps_x_train, ps_x_val, ps_y_train, ps_y_val = sklearn.model_selection.train_test_split(ps_x_train, ps_y_train, test_size=split_valid_size, random_state=1)


<class 'numpy.ndarray'>


In [254]:
print("{0:0.2f}% in training set".format((len(ps_x_train)/len(ps_game_w_team_stats.index))*100))
print("{0:0.2f}% in test set".format((len(ps_x_test)/len(ps_game_w_team_stats.index))*100))
print("{0:0.2f}% in validation set".format((len(ps_x_val)/len(ps_game_w_team_stats.index))*100))

59.84% in training set
20.08% in test set
20.08% in validation set


In [255]:
import sklearn.impute

#Impute with mean all 0 readings
fill_0 = sklearn.impute.SimpleImputer(missing_values=0, strategy="mean")

ps_x_train = fill_0.fit_transform(ps_x_train)
ps_x_test = fill_0.fit_transform(ps_x_test)
ps_x_val = fill_0.fit_transform(ps_x_val)
# TODO : impute incorrect negative values such anything other than (SOS and SRS)

In [256]:
print(len(ps_x_train[0]))

40


In [257]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
dropout = 0.5
model = nn.Sequential(nn.Linear(40, 40),
                     nn.Softmax(dim=1),
                     nn.ReLU(),
                     nn.Dropout(dropout),
                     nn.Linear(40, 40),
                     nn.Softmax(dim=1),
                     nn.ReLU(),
                     nn.Dropout(dropout),
                     nn.Linear(40, 20),
                     nn.Softmax(dim=1),
                     nn.ReLU(),
                     nn.Dropout(dropout),
                     nn.Linear(20, 10),
                     nn.Softmax(dim=1),
                     nn.ReLU(),
                     nn.Dropout(dropout),
                     nn.Linear(10, 1),
                     nn.Sigmoid())
trainset = TensorDataset(torch.from_numpy(ps_x_train).float(), torch.from_numpy(ps_y_train).float())
trainloader = DataLoader(trainset, batch_size=3, shuffle=True)
testset = TensorDataset(torch.from_numpy(ps_x_test).float(), torch.from_numpy(ps_y_test).float())
testloader = DataLoader(testset, batch_size=3, shuffle=True)
valset = TensorDataset(torch.from_numpy(ps_x_val).float(), torch.from_numpy(ps_y_val).float())
valloader = DataLoader(valset, batch_size=3, shuffle=True)
print(next(iter(testloader)))

[tensor([[ 1.5026, -0.8127, -1.0348,  0.9440, -1.1183, -0.1077, -1.1127, -0.2393,
         -0.7499,  0.0825,  1.3138,  0.6320, -0.8084, -0.0917, -0.2072, -0.5913,
          1.4485, -1.3070, -0.7100, -0.8574, -0.6523, -0.5621, -0.8140, -0.6464,
          0.1068, -0.8073, -1.1999,  0.8059,  1.7462,  2.7867,  0.6987, -0.9242,
         -0.4880, -0.2433, -0.3391, -0.1293, -0.3819,  0.1967, -0.1128,  0.2825],
        [ 0.4203,  0.3242, -0.1963, -0.5910,  0.4801,  1.1801,  1.2265,  0.5966,
          0.8056,  1.5437, -0.5475, -1.0058, -0.2449, -0.4487, -1.3252, -0.4797,
         -0.1936, -0.8505, -0.3957,  0.0821, -0.1443,  0.4682,  0.5000, -0.0675,
         -0.7197,  0.8923,  0.2860, -1.1921, -0.7217, -1.3763, -0.6489,  0.5373,
          0.9953, -0.3807, -1.4033,  0.1281,  0.2973, -0.0136, -0.4763, -0.3140],
        [-1.0949,  1.9159,  0.4426,  0.1311, -1.3617,  0.1437, -0.3711,  0.1399,
          0.3467, -0.7537, -0.1964, -0.6685,  0.0373, -1.1872,  0.6528, -1.0004,
         -0.0763,  0.5408

In [258]:
import torch.optim as optim
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [259]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
print ('Using CUDA: {}'.format(use_cuda))
if use_cuda:
    model = model.cuda()

Using CUDA: True


In [260]:
def train(n_epochs, train_loader, val_loader, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    # TODO: fix target tensors always zero for some reason
    valid_loss_min = np.Inf 
    print_loss_count = 40
    cuda_refresh_count = 5
    if use_cuda:
        model = model.cuda()
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        ###################
        # train the model #
        ###################
        model.train()
        train_load_iter = iter(train_loader)
        for i in range(len(train_loader)):
            data, target = next(train_load_iter)
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()    
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
                
            train_loss = train_loss + ((1 / (i + 1)) * (loss.data - train_loss))

        model.eval()
        val_load_iter = iter(val_loader)
        for i in range(len(val_loader)):
            data, target = next(val_load_iter)
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            loss = criterion(output, target)
            valid_loss = valid_loss + ((1 / (i + 1)) * (loss.data - valid_loss))
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        ## save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss

    # return trained model
    return model

In [261]:
model = train(150, trainloader, valloader, model, optimizer, criterion,
                      use_cuda, 'model_nn1.pt')

Epoch: 1 	Training Loss: 0.704656 	Validation Loss: 0.698185
Validation loss decreased (inf --> 0.698185).  Saving model ...
Epoch: 2 	Training Loss: 0.697143 	Validation Loss: 0.696068
Validation loss decreased (0.698185 --> 0.696068).  Saving model ...
Epoch: 3 	Training Loss: 0.697620 	Validation Loss: 0.693812
Validation loss decreased (0.696068 --> 0.693812).  Saving model ...
Epoch: 4 	Training Loss: 0.698152 	Validation Loss: 0.694247
Epoch: 5 	Training Loss: 0.692369 	Validation Loss: 0.693154
Validation loss decreased (0.693812 --> 0.693154).  Saving model ...
Epoch: 6 	Training Loss: 0.694387 	Validation Loss: 0.693158
Epoch: 7 	Training Loss: 0.695123 	Validation Loss: 0.693184
Epoch: 8 	Training Loss: 0.698489 	Validation Loss: 0.693240
Epoch: 9 	Training Loss: 0.697641 	Validation Loss: 0.692544
Validation loss decreased (0.693154 --> 0.692544).  Saving model ...
Epoch: 10 	Training Loss: 0.695664 	Validation Loss: 0.692542
Validation loss decreased (0.692544 --> 0.692542)

Epoch: 86 	Training Loss: 0.594836 	Validation Loss: 0.633745
Epoch: 87 	Training Loss: 0.601713 	Validation Loss: 0.629074
Epoch: 88 	Training Loss: 0.615140 	Validation Loss: 0.618405
Validation loss decreased (0.625624 --> 0.618405).  Saving model ...
Epoch: 89 	Training Loss: 0.623388 	Validation Loss: 0.616477
Validation loss decreased (0.618405 --> 0.616477).  Saving model ...
Epoch: 90 	Training Loss: 0.602842 	Validation Loss: 0.616365
Validation loss decreased (0.616477 --> 0.616365).  Saving model ...
Epoch: 91 	Training Loss: 0.580570 	Validation Loss: 0.615255
Validation loss decreased (0.616365 --> 0.615255).  Saving model ...
Epoch: 92 	Training Loss: 0.595461 	Validation Loss: 0.625886
Epoch: 93 	Training Loss: 0.584783 	Validation Loss: 0.607915
Validation loss decreased (0.615255 --> 0.607915).  Saving model ...
Epoch: 94 	Training Loss: 0.592516 	Validation Loss: 0.609656
Epoch: 95 	Training Loss: 0.577278 	Validation Loss: 0.605090
Validation loss decreased (0.607915

In [262]:
model.load_state_dict(torch.load('model_nn1.pt'))

In [263]:
def test(loader, model, criterion, use_cuda, num_classes = 2):
    if use_cuda:
        model = model.cuda()
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    model.eval()
    load_iter = iter(loader)
    for i in range(len(loader)):
        data, target = next(load_iter)
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (i + 1)) * (loss.data - test_loss))
        # compare predictions to true label
        for j, tensor in enumerate(output):
            if (tensor.item() > .5 and target[j] == 1) or (tensor.item() <= .5 and target[j] == 0):
                correct += 1
        total += data.size(0)
       
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [264]:
test(testloader, model, criterion, use_cuda)

Test Loss: 0.665929


Test Accuracy: 64% (64/100)


In [265]:
# Setup This years bracket regions
# TODO: automate this with the data received from the scraper
school_names_south = [
    # south region
    ('Virginia', 1),('Gardner-Webb', 16),
    ('Ole Miss', 8), ('Oklahoma', 9),
    ('Wisconsin',5), ('Oregon', 12),
    ('Kansas State',4), ('UC-Irvine', 13),
    ('Villanova', 6), ("Saint Mary's", 11),
    ('Purdue',3), ('Old Dominion',14),
    ('Cincinnati',7),('Iowa',10),
    ('Tennessee',2), ('Colgate',15)
    ]
school_names_west = [
    # west region
    ('Gonzaga', 1),('North Carolina Central',16), #or 'Texas Southern',
    ('Syracuse', 8),('Baylor', 9),
    ('Marquette',5), ('Murray State', 12),
    ('Florida State',4), ('Vermont',13),
    ('Buffalo',6),('San Diego State',11), # or
    ('Texas Tech', 3),('Northern Kentucky', 14),
    ('Nevada',7),('Florida',10),
    ('Michigan',2),('Montana',15)
    ]
school_names_east = [
    # east region
    ('Duke',1),('Long Island University',16), # or 'Radford',
    ('VCU',8), ('UCF',9),
    ('Mississippi State',5), ('Liberty',12),
    ('Virginia Tech',4), ('Saint Louis',13),
    ('Maryland',6), ('St. Bonaventure',11), # or 'UCLA',
    ('LSU',3), ('Yale',14),
    ('Louisville',7), ('Minnesota',10),
    ('Michigan State', 2), ('Bradley',15)
    ]
school_names_midwest = [
    # mid-west region
    ('UNC', 1),('Iona', 16),
    ('Utah State', 8), ('Washington', 9),
    ('Auburn',5), ('New Mexico State', 12),
    ('Kansas',4), ('Northeastern', 13),
    ('Iowa State', 6), ('Ohio State', 11),
    ('Houston',3), ('Georgia State',14),
    ('Wofford',7),('Seton Hall',10),
    ('Kentucky',2), ('Abilene Christian',15)
    ]
'''
2018 Bracket
school_names_south = [
    # south region
    ('Virginia', 1),('Maryland-Baltimore County', 16),
    ('Creighton', 8), ('Kansas State', 9),
    ('Kentucky',5), ('Davidson', 12),
    ('Arizona',4), ('Buffalo', 13),
    ('Miami (FL)', 6), ('Loyola (IL)', 11),
    ('Tennessee',3), ('Wright State',14),
    ('Nevada',7),('Texas',10),
    ('Cincinnati',2), ('Georgia State',15)
    ]
school_names_west = [
    # west region
    ('Xavier', 1),('North Carolina Central',16), #or 'Texas Southern',
    ('Missouri', 8),('Florida State', 9),
    ('Ohio State',5), ('South Dakota State', 12),
    ('Gonzaga',4), ('North Carolina-Greensboro',13),
    ('Houston',6),('San Diego State',11),
    ('Michigan', 3),('Montana', 14),
    ('Texas A&M',7),('Providence',10),
    ('North Carolina',2),('Lipscomb',15)
    ]
school_names_east = [
    # east region
    ('Villanova',1),('Long Island University',16), # or 'Radford',
    ('Virginia Tech',8), ('Alabama',9),
    ('West Virginia',5), ('Murray State',12),
    ('Wichita State',4), ('Marshall',13),
    ('Florida',6), ('St. Bonaventure',11), # or 'UCLA',
    ('Texas Tech',3), ('Stephen F. Austin',14),
    ('Arkansas',7), ('Butler',10),
    ('Purdue', 2), ('Cal State Fullerton',15)
    ]
school_names_midwest = [
    # mid-west region
    ('Kansas', 1), ('Pennsylvania',16),
    ('Seton Hall', 8), ('North Carolina State',9),
    ('Clemson', 5), ('New Mexico State',12),
    ('Auburn',4), ('College of Charleston',13),
    ('Texas Christian',6), ('Arizona State',11), # or 'Syracuse',
    ('Michigan State',3), ('Bucknell',14),
    ('Rhode Island',7), ('Oklahoma',10),
    ('Duke', 2), ('Iona' ,15) 
    ]
'''

"\n2018 Bracket\nschool_names_south = [\n    # south region\n    ('Virginia', 1),('Maryland-Baltimore County', 16),\n    ('Creighton', 8), ('Kansas State', 9),\n    ('Kentucky',5), ('Davidson', 12),\n    ('Arizona',4), ('Buffalo', 13),\n    ('Miami (FL)', 6), ('Loyola (IL)', 11),\n    ('Tennessee',3), ('Wright State',14),\n    ('Nevada',7),('Texas',10),\n    ('Cincinnati',2), ('Georgia State',15)\n    ]\nschool_names_west = [\n    # west region\n    ('Xavier', 1),('North Carolina Central',16), #or 'Texas Southern',\n    ('Missouri', 8),('Florida State', 9),\n    ('Ohio State',5), ('South Dakota State', 12),\n    ('Gonzaga',4), ('North Carolina-Greensboro',13),\n    ('Houston',6),('San Diego State',11),\n    ('Michigan', 3),('Montana', 14),\n    ('Texas A&M',7),('Providence',10),\n    ('North Carolina',2),('Lipscomb',15)\n    ]\nschool_names_east = [\n    # east region\n    ('Villanova',1),('Long Island University',16), # or 'Radford',\n    ('Virginia Tech',8), ('Alabama',9),\n    ('Wes

In [266]:
'''
    Methods to add evaluating the predicted winners of matchups and subbrackets (A region or Final Four)
    To change the predictive model used, just change the model handed to "evaluate_winner(schools,sub_bracket_name, model)"
    found later in the notebook
'''
def get_matchup_winners(matchup_stats, schools, model, post_season, use_cuda):
    x_tourney = matchup_stats[shared.ps_feature_col_names].values
    x_tourney = torch.from_numpy(x_tourney).float()
    # print(x_tourney)
    if use_cuda:
        x_tourney = x_tourney.cuda()
    y_tourney = model(x_tourney)
    print(y_tourney)
    i = 0
    winners = []
    for y_val in y_tourney:
        t1_name, t1_seed = schools[i]
        t2_name, t2_seed = schools[i + 1]
        print('t1 win?: {}'.format(y_val.item()))
        t1_won = y_val.item() > .5
        print(t1_name,t1_seed,' vs. ', t2_name,t2_seed,'(team 1 won=', t1_won,')')
        if(t1_won):
            winners.append((t1_name,t1_seed))
        else:
            winners.append((t2_name, t2_seed))
        i = i + 2
    return winners
def evaluate_winner(schools,sub_bracket_name, model, use_cuda):        
    remaining_teams = schools
    i = 1
    while(len(remaining_teams) > 1):
        #Add a random factor
        rand = random.randrange(0,1)
        post_season_stats = True
        print("---",sub_bracket_name," round ",i,"---")
        matchup_stats = shared.get_matchups_stats(df_school, remaining_teams, post_season_stats)
        remaining_teams = get_matchup_winners(matchup_stats,remaining_teams, model, post_season_stats, use_cuda)
        i = i + 1
    winner = remaining_teams[0]
    print('Winner of ',sub_bracket_name,':',winner)
    return winner

In [231]:
# Get predicted final four

final_four = [evaluate_winner(school_names_south, "South",model, use_cuda), evaluate_winner(school_names_west,"West",model, use_cuda),
              evaluate_winner(school_names_east, "East", model, use_cuda), evaluate_winner(school_names_midwest, "MidWest",model, use_cuda)]

--- South  round  1 ---
tensor([[0.8898],
        [0.8861],
        [0.8884],
        [0.8883],
        [0.8883],
        [0.8885],
        [0.8876],
        [0.8890]], device='cuda:0', grad_fn=<SigmoidBackward>)
t1 win?: 0.8898149132728577
Virginia 1  vs.  Gardner-Webb 16 (team 1 won= True )
t1 win?: 0.8860703706741333
Ole Miss 8  vs.  Oklahoma 9 (team 1 won= True )
t1 win?: 0.8883756399154663
Wisconsin 5  vs.  Oregon 12 (team 1 won= True )
t1 win?: 0.8882895112037659
Kansas State 4  vs.  UC-Irvine 13 (team 1 won= True )
t1 win?: 0.8882968425750732
Villanova 6  vs.  Saint Mary's 11 (team 1 won= True )
t1 win?: 0.8884812593460083
Purdue 3  vs.  Old Dominion 14 (team 1 won= True )
t1 win?: 0.8876286149024963
Cincinnati 7  vs.  Iowa 10 (team 1 won= True )
t1 win?: 0.888972818851471
Tennessee 2  vs.  Colgate 15 (team 1 won= True )
--- South  round  2 ---
tensor([[0.8881],
        [0.8880],
        [0.8822],
        [0.8844]], device='cuda:0', grad_fn=<SigmoidBackward>)
t1 win?: 0.88813686

In [267]:
final_four

[('Virginia', 1), ('Gonzaga', 1), ('Duke', 1), ('UNC', 1)]

In [268]:
champ = evaluate_winner(final_four, "FinalFour", model, use_cuda)

--- FinalFour  round  1 ---
tensor([[0.7630],
        [0.7630]], device='cuda:0', grad_fn=<SigmoidBackward>)
t1 win?: 0.7630281448364258
Virginia 1  vs.  Gonzaga 1 (team 1 won= True )
t1 win?: 0.7630281448364258
Duke 1  vs.  UNC 1 (team 1 won= True )
--- FinalFour  round  2 ---
tensor([[0.7630]], device='cuda:0', grad_fn=<SigmoidBackward>)
t1 win?: 0.7630282044410706
Virginia 1  vs.  Duke 1 (team 1 won= True )
Winner of  FinalFour : ('Virginia', 1)
